# Amazon SageMaker Batch Transform: Associate prediction results with their corresponding input records


---

This notebook's CI test result for us-west-2 is as follows. CI test results in other regions can be found at the end of the notebook. 

![This us-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/us-west-2/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with high level SDK.ipynb)

---

_**Use SageMaker's XGBoost to train a binary classification model and for a list of tumors in batch file, predict if each is malignant**_

_**It also shows how to use the input output joining / filter feature in Batch transform in details**_

---



## Background
This purpose of this notebook is to train a model using SageMaker's XGBoost and UCI's breast cancer diagnostic data set to illustrate at how to run batch inferences and how to use the Batch Transform I/O join feature. UCI's breast cancer diagnostic data set is available at https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29. The data set is also available on Kaggle at https://www.kaggle.com/uciml/breast-cancer-wisconsin-data. The purpose here is to use this data set to build a predictve model of whether a breast mass image indicates benign or malignant tumor. 



---

## Setup

Let's start by specifying:

* The SageMaker role arn used to give training and batch transform access to your data. The snippet below will use the same role used by your SageMaker notebook instance. Otherwise, specify the full ARN of a role with the SageMakerFullAccess policy attached.
* The S3 bucket that you want to use for training and storing model objects.

In [3]:
!pip3 install -U sagemaker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 48.5 MB/s eta 0:00:00:00:01
  Attempting uninstall: mock
    Found existing installation: mock 5.1.0
    Uninstalling mock-5.1.0:
      Successfully uninstalled mock-5.1.0
  Attempting uninstall: botocore
    Found existing installation: botocore 1.34.84
    Uninstalling botocore-1.34.84:
      Successfully uninstalled botocore-1.34.84
  Attempting uninstall: boto3
    Found existing installation: boto3 1.34.84
    Uninstalling boto3-1.34.84:
      Successfully uninstalled boto3-1.34.84
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.215.0
    Uninstalling sagemaker-2.215.0:
      Successfully uninstalled sagemaker-2.215.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.32.84 requires botoc

In [2]:
import os
import boto3
import sagemaker

role = sagemaker.get_execution_role()
sess = sagemaker.Session()
region = sess.boto_region_name

bucket = sess.default_bucket()
prefix = "DEMO-breast-cancer-prediction-xgboost-highlevel"

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


---
## Data sources

> Dua, D. and Graff, C. (2019). UCI Machine Learning Repository [http://archive.ics.uci.edu/ml]. Irvine, CA: University of California, School of Information and Computer Science.

> Breast Cancer Wisconsin (Diagnostic) Data Set [https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic)].

> _Also see:_ Breast Cancer Wisconsin (Diagnostic) Data Set [https://www.kaggle.com/uciml/breast-cancer-wisconsin-data].

## Data preparation


Let's download the data and save it in the local folder with the name data.csv and take a look at it.

In [3]:
import pandas as pd
import numpy as np
# explore how to perform this step if the data source was in kaggle instead
s3 = boto3.client("s3")

filename = "wdbc.csv"
s3.download_file(
    f"sagemaker-example-files-prod-{region}", "datasets/tabular/breast_cancer/wdbc.csv", filename
)
data = pd.read_csv(filename, header=None)

# specify columns extracted from wbdc.names
data.columns = [
    "id",
    "diagnosis",
    "radius_mean",
    "texture_mean",
    "perimeter_mean",
    "area_mean",
    "smoothness_mean",
    "compactness_mean",
    "concavity_mean",
    "concave points_mean",
    "symmetry_mean",
    "fractal_dimension_mean",
    "radius_se",
    "texture_se",
    "perimeter_se",
    "area_se",
    "smoothness_se",
    "compactness_se",
    "concavity_se",
    "concave points_se",
    "symmetry_se",
    "fractal_dimension_se",
    "radius_worst",
    "texture_worst",
    "perimeter_worst",
    "area_worst",
    "smoothness_worst",
    "compactness_worst",
    "concavity_worst",
    "concave points_worst",
    "symmetry_worst",
    "fractal_dimension_worst",
]

# save the data
data.to_csv("data.csv", sep=",", index=False)

data.sample(8)

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
229,881861,M,12.830,22.33,85.26,503.2,0.10880,0.17990,0.16950,0.06861,...,15.200,30.15,105.30,706.0,0.17770,0.53430,0.62820,0.19770,0.3407,0.12430
33,854002,M,19.270,26.47,127.90,1162.0,0.09401,0.17190,0.16570,0.07593,...,24.150,30.90,161.40,1813.0,0.15090,0.65900,0.60910,0.17850,0.3672,0.11230
123,865432,B,14.500,10.89,94.28,640.7,0.11010,0.10990,0.08842,0.05778,...,15.700,15.98,102.80,745.5,0.13130,0.17880,0.25600,0.12210,0.2889,0.08006
523,917896,B,13.710,18.68,88.73,571.0,0.09916,0.10700,0.05385,0.03783,...,15.110,25.63,99.43,701.9,0.14250,0.25660,0.19350,0.12840,0.2849,0.09031
91,861799,M,15.370,22.76,100.20,728.2,0.09200,0.10360,0.11220,0.07483,...,16.430,25.84,107.50,830.9,0.12570,0.19970,0.28460,0.14760,0.2556,0.06828
191,875093,B,12.770,21.41,82.02,507.4,0.08749,0.06601,0.03112,0.02864,...,13.750,23.50,89.04,579.5,0.09388,0.08978,0.05186,0.04773,0.2179,0.06871
217,8811779,B,10.200,17.48,65.05,321.2,0.08054,0.05907,0.05774,0.01071,...,11.480,24.47,75.40,403.7,0.09527,0.13970,0.19250,0.03571,0.2868,0.07809
175,872113,B,8.671,14.45,54.42,227.2,0.09138,0.04276,0.00000,0.00000,...,9.262,17.04,58.36,259.2,0.11620,0.07057,0.00000,0.00000,0.2592,0.07848


#### Key observations:
* The data has 569 observations and 32 columns.
* The first field is the 'id' attribute that we will want to drop before batch inference and add to the final inference output next to the probability of malignancy.
* Second field, 'diagnosis', is an indicator of the actual diagnosis ('M' = Malignant; 'B' = Benign).
* There are 30 other numeric features that we will use for training and inferencing.

Let's replace the M/B diagnosis with a 1/0 boolean value. 

In [4]:
data["diagnosis"] = data["diagnosis"].apply(lambda x: ((x == "M")) + 0)
data.sample(8)

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
331,896864,0,12.980,19.35,84.52,514.0,0.09579,0.11250,0.07107,0.02950,...,14.42,21.95,99.21,634.3,0.1288,0.3253,0.34390,0.09858,0.3596,0.09166
396,90401601,0,13.510,18.89,88.10,558.1,0.10590,0.11470,0.08580,0.05381,...,14.80,27.20,97.33,675.2,0.1428,0.2570,0.34380,0.14530,0.2666,0.07686
142,869218,0,11.430,17.31,73.66,398.0,0.10920,0.09486,0.02031,0.01861,...,12.78,26.76,82.66,503.0,0.1413,0.1792,0.07708,0.06402,0.2584,0.08096
562,925622,1,15.220,30.62,103.40,716.9,0.10480,0.20870,0.25500,0.09429,...,17.52,42.79,128.70,915.0,0.1417,0.7917,1.17000,0.23560,0.4089,0.14090
391,903483,0,8.734,16.84,55.27,234.3,0.10390,0.07428,0.00000,0.00000,...,10.17,22.80,64.01,317.0,0.1460,0.1310,0.00000,0.00000,0.2445,0.08865
486,913102,0,14.640,16.85,94.21,666.0,0.08641,0.06698,0.05192,0.02791,...,16.46,25.44,106.00,831.0,0.1142,0.2070,0.24370,0.07828,0.2455,0.06596
487,913505,1,19.440,18.82,128.10,1167.0,0.10890,0.14480,0.22560,0.11940,...,23.96,30.39,153.90,1740.0,0.1514,0.3725,0.59360,0.20600,0.3266,0.09009
34,854039,1,16.130,17.88,107.00,807.2,0.10400,0.15590,0.13540,0.07752,...,20.21,27.26,132.70,1261.0,0.1446,0.5804,0.52740,0.18640,0.4270,0.12330


Let's split the data as follows: 80% for training, 10% for validation and let's set 10% aside for our batch inference job. In addition, let's drop the 'id' field on the training set and validation set as 'id' is not a training feature. For our batch set however, we keep the 'id' feature. We'll want to filter it out prior to running our inferences so that the input data features match the ones of training set and then ultimately, we'll want to join it with inference result. We are however dropping the diagnosis attribute for the batch set since this is what we'll try to predict.

In [5]:
# data split in three sets, training, validation and batch inference
rand_split = np.random.rand(len(data))
train_list = rand_split < 0.8
val_list = (rand_split >= 0.8) & (rand_split < 0.9)
batch_list = rand_split >= 0.9

data_train = data[train_list].drop(["id"], axis=1)
data_val = data[val_list].drop(["id"], axis=1)
data_batch = data[batch_list].drop(["diagnosis"], axis=1)
data_batch_noID = data_batch.drop(["id"], axis=1)

In [6]:
rand_split = np.random.rand(10)
train_list = rand_split < 0.8
val_list = (rand_split >= 0.8) & (rand_split < 0.9)
batch_list = rand_split >= 0.9
print(rand_split)
print (train_list)
print (val_list)
print (batch_list)

[0.93467502 0.12969392 0.88028553 0.12946166 0.95573246 0.19313113
 0.61239973 0.14265352 0.01244423 0.2236181 ]
[False  True False  True False  True  True  True  True  True]
[False False  True False False False False False False False]
[ True False False False  True False False False False False]


Let's upload those data sets in S3

In [7]:
train_file = "train_data.csv"
data_train.to_csv(train_file, index=False, header=False)
sess.upload_data(train_file, key_prefix="{}/train".format(prefix))

validation_file = "validation_data.csv"
data_val.to_csv(validation_file, index=False, header=False)
sess.upload_data(validation_file, key_prefix="{}/validation".format(prefix))

batch_file = "batch_data.csv"
data_batch.to_csv(batch_file, index=False, header=False)
sess.upload_data(batch_file, key_prefix="{}/batch".format(prefix))

batch_file_noID = "batch_data_noID.csv"
data_batch_noID.to_csv(batch_file_noID, index=False, header=False)
sess.upload_data(batch_file_noID, key_prefix="{}/batch".format(prefix))

's3://sagemaker-us-east-1-284926681852/DEMO-breast-cancer-prediction-xgboost-highlevel/batch/batch_data_noID.csv'

---

## Training job and model creation

The below cell uses the [SageMaker Python SDK](https://github.com/aws/sagemaker-python-sdk) to kick off the training job using both our training set and validation set. Not that the objective is set to 'binary:logistic' which trains a model to output a probability between 0 and 1 (here the probability of a tumor being malignant).

In [8]:
%%time
from time import gmtime, strftime

job_name = "xgb-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
output_location = "s3://{}/{}/output/{}".format(bucket, prefix, job_name)
image = sagemaker.image_uris.retrieve(
    framework="xgboost", region=boto3.Session().region_name, version="1.7-1"
)

sm_estimator = sagemaker.estimator.Estimator(
    image,
    role,
    instance_count=1,
    instance_type="ml.m5.xlarge",
    volume_size=50,
    input_mode="File",
    output_path=output_location,
    sagemaker_session=sess,
)

sm_estimator.set_hyperparameters(
    objective="binary:logistic",
    max_depth=5,
    eta=0.2,
    gamma=4,
    min_child_weight=6,
    subsample=0.8,
    verbosity=0,
    num_round=100,
)

train_data = sagemaker.inputs.TrainingInput(
    "s3://{}/{}/train".format(bucket, prefix),
    distribution="FullyReplicated",
    content_type="text/csv",
    s3_data_type="S3Prefix",
)
validation_data = sagemaker.inputs.TrainingInput(
    "s3://{}/{}/validation".format(bucket, prefix),
    distribution="FullyReplicated",
    content_type="text/csv",
    s3_data_type="S3Prefix",
)
data_channels = {"train": train_data, "validation": validation_data}

# Start training by calling the fit method in the estimator
sm_estimator.fit(inputs=data_channels, job_name=job_name, logs=True)

INFO:sagemaker:Creating training-job with name: xgb-2024-10-01-02-34-42


2024-10-01 02:34:44 Starting - Starting the training job...
2024-10-01 02:34:57 Starting - Preparing the instances for training...
2024-10-01 02:35:30 Downloading - Downloading input data...
2024-10-01 02:36:00 Downloading - Downloading the training image...
2024-10-01 02:36:46 Training - Training image download completed. Training in progress...[2024-10-01 02:36:56.568 ip-10-0-245-124.ec2.internal:7 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2024-10-01 02:36:56.591 ip-10-0-245-124.ec2.internal:7 INFO profiler_config_parser.py:111] User has disabled profiler.
[2024-10-01:02:36:56:INFO] Imported framework sagemaker_xgboost_container.training
[2024-10-01:02:36:56:INFO] Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
[2024-10-01:02:36:56:INFO] No GPUs detected (normal if no gpus installed)
[2024-10-01:02:36:57:INFO] Running XGBoost Sagemaker in algorithm mode
[2024-10-01:02:36:57:INFO] Determined 0 GPU(s) available on the inst

---

## Batch Transform

In SageMaker Batch Transform, we introduced 3 new attributes - __input_filter__, __join_source__ and __output_filter__. In the below cell, we use the [SageMaker Python SDK](https://github.com/aws/sagemaker-python-sdk) to kick-off several Batch Transform jobs using different configurations of these 3 new attributes. Please refer to [this page](https://docs.aws.amazon.com/sagemaker/latest/dg/batch-transform-data-processing.html) to learn more about how to use them.




#### 1. Create a transform job with the default configurations
Let's first skip these 3 new attributes and inspect the inference results. We'll use it as a baseline to compare to the results with data processing.

In [9]:
%%time

sm_transformer = sm_estimator.transformer(1, "ml.m4.xlarge")

# start a transform job
input_location = "s3://{}/{}/batch/{}".format(
    bucket, prefix, batch_file_noID
)  # use input data without ID column
sm_transformer.transform(input_location, content_type="text/csv", split_type="Line")
sm_transformer.wait()

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2024-10-01-02-38-04-249
INFO:sagemaker:Creating transform job with name: sagemaker-xgboost-2024-10-01-02-38-05-184


..........................................
[2024-10-01:02:45:02:INFO] No GPUs detected (normal if no gpus installed)
[2024-10-01:02:45:02:INFO] No GPUs detected (normal if no gpus installed)
[2024-10-01:02:45:02:INFO] nginx config: 
[2024-10-01:02:45:02:INFO] No GPUs detected (normal if no gpus installed)
[2024-10-01:02:45:02:INFO] No GPUs detected (normal if no gpus installed)
[2024-10-01:02:45:02:INFO] nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr;
worker_rlimit_nofile 4096;
events {
  worker_connections 2048;
}
http {
  include /etc/nginx/mime.types;
  default_type application/octet-stream;
  access_log /dev/stdout combined;
  upstream gunicorn {
    server unix:/tmp/gunicorn.sock;
  }
  server {
    listen 8080 deferred;
    client_max_body_size 0;
    keepalive_timeout 3;
    location ~ ^/(ping|invocations|execution-parameters) {
      proxy_set_header X-Forwarded-For $proxy_add_x_forwarded_for;
      proxy_set_header Host $http_host;

Let's inspect the output of the Batch Transform job in S3. It should show the list probabilities of tumors being malignant.

In [10]:
import re


def get_csv_output_from_s3(s3uri, batch_file):
    file_name = "{}.out".format(batch_file)
    match = re.match("s3://([^/]+)/(.*)", "{}/{}".format(s3uri, file_name))
    output_bucket, output_prefix = match.group(1), match.group(2)
    s3.download_file(output_bucket, output_prefix, file_name)
    return pd.read_csv(file_name, sep=",", header=None)

In [11]:
output_df = get_csv_output_from_s3(sm_transformer.output_path, batch_file_noID)
output_df.head(8)

,0
0,0.993822
1,0.994906
2,0.063616
3,0.992156
4,0.990236
5,0.042464
6,0.994906
7,0.028461


#### 2. Join the input and the prediction results 
Now, let's associate the prediction results with their corresponding input records. We can also use the __input_filter__ to exclude the ID column easily and there's no need to have a separate file in S3.

* Set __input_filter__ to "$[1:]": indicates that we are excluding column 0 (the 'ID') before processing the inferences and keeping everything from column 1 to the last column (all the features or predictors)  
  
  
* Set __join_source__ to "Input": indicates our desire to join the input data with the inference results  

* Leave __output_filter__ to default ('$'), indicating that the joined input and inference results be will saved as output.

In [20]:
# content_type / accept and split_type / assemble_with are required to use IO joining feature
sm_transformer.assemble_with = "Line"
sm_transformer.accept = "text/csv"

# start a transform job
input_location = "s3://{}/{}/batch/{}".format(
    bucket, prefix, batch_file
)  # use input data with ID column cause InputFilter will filter it out
sm_transformer.transform(
    input_location,
    split_type="Line",
    content_type="text/csv",
    input_filter="$[1:]",
    join_source="Input",
)
sm_transformer.wait()

INFO:sagemaker:Creating transform job with name: sagemaker-xgboost-2024-10-01-01-26-32-490


...........................................[2024-10-01:01:33:45:INFO] No GPUs detected (normal if no gpus installed)
[2024-10-01:01:33:45:INFO] No GPUs detected (normal if no gpus installed)
[2024-10-01:01:33:45:INFO] nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr;
worker_rlimit_nofile 4096;
events {
  worker_connections 2048;
}
http {
  include /etc/nginx/mime.types;
  default_type application/octet-stream;
  access_log /dev/stdout combined;
  upstream gunicorn {
    server unix:/tmp/gunicorn.sock;
  }
  server {
    listen 8080 deferred;
    client_max_body_size 0;
    keepalive_timeout 3;
    location ~ ^/(ping|invocations|execution-parameters) {
      proxy_set_header X-Forwarded-For $proxy_add_x_forwarded_for;
      proxy_set_header Host $http_host;
      proxy_redirect off;
      proxy_read_timeout 60s;
      proxy_pass http://gunicorn;
    }
    location / {
      return 404 "{}";
    }
  }
}
[2024-10-01 01:33:45 +0000] [19] [INFO] S

Let's inspect the output of the Batch Transform job in S3. It should show the list of tumors identified by their original feature columns and their corresponding probabilities of being malignant.

In [21]:
output_df = get_csv_output_from_s3(sm_transformer.output_path, batch_file)
output_df.head(8)

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,84348301,11.420,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.105200,0.2597,...,26.50,98.87,567.7,0.2098,0.8663,0.68690,0.25750,0.6638,0.17300,0.598975
1,844359,18.250,19.98,119.60,1040.0,0.09463,0.10900,0.11270,0.074000,0.1794,...,27.66,153.20,1606.0,0.1442,0.2576,0.37840,0.19320,0.3063,0.08368,0.993214
2,849014,19.810,22.15,130.00,1260.0,0.09831,0.10270,0.14790,0.094980,0.1582,...,30.88,186.80,2398.0,0.1512,0.3150,0.53720,0.23880,0.2768,0.07615,0.993253
3,8510824,9.504,12.44,60.34,273.9,0.10240,0.06492,0.02956,0.020760,0.1815,...,15.66,65.13,314.9,0.1324,0.1148,0.08867,0.06227,0.2450,0.07773,0.005575
4,852552,16.650,21.38,110.00,904.6,0.11210,0.14570,0.15250,0.091700,0.1995,...,31.56,177.00,2215.0,0.1805,0.3578,0.46950,0.20950,0.3613,0.09564,0.994544
5,852631,17.140,16.40,116.00,912.7,0.11860,0.22760,0.22290,0.140100,0.3040,...,21.40,152.40,1461.0,0.1545,0.3949,0.38530,0.25500,0.4066,0.10590,0.952166
6,857010,18.650,17.60,123.70,1076.0,0.10990,0.16860,0.19740,0.100900,0.1907,...,21.32,150.60,1567.0,0.1679,0.5090,0.73450,0.23780,0.3799,0.09185,0.952166
7,85713702,8.196,16.84,51.71,201.9,0.08600,0.05943,0.01588,0.005917,0.1769,...,21.96,57.26,242.2,0.1297,0.1357,0.06880,0.02564,0.3105,0.07409,0.006815


#### 3. Update the output filter to keep only ID and prediction results
Let's change __output_filter__ to "$[0,-1]", indicating that when presenting the output, we only want to keep column 0 (the 'ID') and the last column (the inference result i.e. the probability of a given tumor to be malignant)

In [12]:
# start another transform job
sm_transformer.transform(
    input_location,
    split_type="Line",
    content_type="text/csv",
    input_filter="$[1:]",
    join_source="Input",
    output_filter="$[0,-1]",
)
sm_transformer.wait()

Now, let's inspect the output of the Batch Transform job in S3 again. It should show 2 columns: the ID and their corresponding probabilities of being malignant.

In [13]:
output_df = get_csv_output_from_s3(sm_transformer.output_path, batch_file)
output_df.head(8)

create_model(role=role, image_uri=XGBOOST_IMAGE)In summary, we can use newly introduced 3 attributes - __input_filter__, __join_source__, __output_filter__ to 
1. Filter / select useful features from the input dataset. e.g. exclude ID columns.
2. Associate the prediction results with their corresponding input records.
3. Filter the original or joined results before saving to S3. e.g. keep ID and probability columns only.

## Upload the Sagemaker Model created during our training job to the Sagemaker Model Registry

In [16]:
sagemaker = boto3.client("sagemaker")

#model_name = "sagemaker-xgboost-2023-08-22-05-28-37-903"
model_name='xgb-2024-10-01-02-34-42'
print(model_name)


info = sagemaker.describe_training_job(TrainingJobName=model_name)
model_data = info["ModelArtifacts"]["S3ModelArtifacts"]

primary_container = {"Image": image, "ModelDataUrl": model_data}

# Save our model to the Sagemaker Model Registry
create_model_response = sagemaker.create_model(
    ModelName=model_name, ExecutionRoleArn=role, PrimaryContainer=primary_container
)

print(create_model_response["ModelArn"])

xgb-2024-10-01-02-34-42
arn:aws:sagemaker:us-east-1:284926681852:model/xgb-2024-10-01-02-34-42


In [17]:
# Inspect Training Job Details
info

{'TrainingJobName': 'xgb-2024-10-01-02-34-42',
 'TrainingJobArn': 'arn:aws:sagemaker:us-east-1:284926681852:training-job/xgb-2024-10-01-02-34-42',
 'ModelArtifacts': {'S3ModelArtifacts': 's3://sagemaker-us-east-1-284926681852/DEMO-breast-cancer-prediction-xgboost-highlevel/output/xgb-2024-10-01-02-34-42/xgb-2024-10-01-02-34-42/output/model.tar.gz'},
 'TrainingJobStatus': 'Completed',
 'SecondaryStatus': 'Completed',
 'HyperParameters': {'eta': '0.2',
  'gamma': '4',
  'max_depth': '5',
  'min_child_weight': '6',
  'num_round': '100',
  'objective': 'binary:logistic',
  'subsample': '0.8',
  'verbosity': '0'},
 'AlgorithmSpecification': {'TrainingImage': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.7-1',
  'TrainingInputMode': 'File',
  'MetricDefinitions': [{'Name': 'train:mae',
    'Regex': '.*\\[[0-9]+\\].*#011train-mae:([-+]?[0-9]*\\.?[0-9]+(?:[eE][-+]?[0-9]+)?).*'},
   {'Name': 'validation:aucpr',
    'Regex': '.*\\[[0-9]+\\].*#011validation-aucpr:([-+]?[0-9]*\

In [18]:
# Create Endpoint Configuration


# Create an endpoint config name. Here we create one based on the date  
# so it we can search endpoints based on creation time.
endpoint_config_name = 'lab4-1-endpoint-config' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())                            
                            
instance_type = 'ml.m5.xlarge'

endpoint_config_response = sagemaker.create_endpoint_config(
    EndpointConfigName=endpoint_config_name, # You will specify this name in a CreateEndpoint request.
    # List of ProductionVariant objects, one for each model that you want to host at this endpoint.
    ProductionVariants=[
        {
            "VariantName": "variant1", # The name of the production variant.
            "ModelName": model_name, 
            "InstanceType": instance_type, # Specify the compute instance type.
            "InitialInstanceCount": 1 # Number of instances to launch initially.
        }
    ]
)

print(f"Created EndpointConfig: {endpoint_config_response['EndpointConfigArn']}")


Created EndpointConfig: arn:aws:sagemaker:us-east-1:284926681852:endpoint-config/lab4-1-endpoint-config2024-10-01-02-53-29


In [19]:
# Deploy our model to real-time endpoint

endpoint_name = 'lab4-1-endpoint' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())                            


create_endpoint_response = sagemaker.create_endpoint(
                                            EndpointName=endpoint_name, 
                                            EndpointConfigName=endpoint_config_name) 

In [32]:
# Wait for endpoint to spin up

sagemaker.describe_endpoint(EndpointName=endpoint_name)

while True:
    print("Getting Job Status")
    res = sagemaker.describe_endpoint(EndpointName=endpoint_name)
    state = res["EndpointStatus"]
    
    if state == "InService":
        print("Endpoint in Service")
        break
    elif state == "Creating":
        print("Endpoint still creating...")
        sleep(60)
    else:
        print("Endpoint Creation Error - Check Sagemaker Console")
        break

Getting Job Status
Endpoint in Service


In [43]:
# Invoke Endpoint

sagemaker_runtime = boto3.client("sagemaker-runtime", region_name=region)

response = sagemaker_runtime.invoke_endpoint(
                            EndpointName=endpoint_name,
                            ContentType='text/csv',
                            Body=data_batch_noID.to_csv(header=None, index=False).strip('\n').split('\n')[0]
                            )
print(response['Body'].read().decode('utf-8'))

0.9781807065010071



In [44]:
# Examine Response Body

response

{'ResponseMetadata': {'RequestId': '94c8728d-4c67-40d4-88ad-1ef5eaa70ae1',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '94c8728d-4c67-40d4-88ad-1ef5eaa70ae1',
   'x-amzn-invoked-production-variant': 'variant1',
   'date': 'Tue, 22 Aug 2023 07:02:46 GMT',
   'content-type': 'text/csv; charset=utf-8',
   'content-length': '19',
   'connection': 'keep-alive'},
  'RetryAttempts': 0},
 'ContentType': 'text/csv; charset=utf-8',
 'InvokedProductionVariant': 'variant1',
 'Body': <botocore.response.StreamingBody at 0x7ffb93f543d0>}

In [45]:
# Delete Endpoint

sagemaker.delete_endpoint(EndpointName=endpoint_name)

{'ResponseMetadata': {'RequestId': 'c2b381e2-33a0-4b70-8732-fbaa44fab5a0',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'c2b381e2-33a0-4b70-8732-fbaa44fab5a0',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Tue, 22 Aug 2023 07:10:18 GMT'},
  'RetryAttempts': 0}}

# Model Group

In [21]:
# Create a SageMaker client
sagemaker = boto3.client('sagemaker')

# Define model package group name and description
model_package_group_name = "xgb-breastcancer-detection"
model_package_group_description = "Model group for XGBoost models for breast cancer detection."

# Create the model package group
response = sagemaker.create_model_package_group(
    ModelPackageGroupName=model_package_group_name,
    ModelPackageGroupDescription=model_package_group_description
)

print(f"Model Package Group ARN: {response['ModelPackageGroupArn']}")


Model Package Group ARN: arn:aws:sagemaker:us-east-1:284926681852:model-package-group/xgb-breastcancer-detection


In [26]:
# Define model package group name and the exact model data file
model_data = 's3://sagemaker-us-east-1-284926681852/DEMO-breast-cancer-prediction-xgboost-highlevel/output/xgb-2024-10-01-02-34-42/xgb-2024-10-01-02-34-42/output/model.tar.gz'

# Create model package for the XGBoost model
create_model_package_response = sagemaker.create_model_package(
    ModelPackageGroupName=model_package_group_name,
    ModelPackageDescription="Versioned XGBoost model for breast cancer detection.",
    ModelApprovalStatus="Approved",  # You can change to "PendingManualApproval" if manual approval is needed
    InferenceSpecification={
        "Containers": [
            {
                "Image": image,  # Replace with your image
                "ModelDataUrl": model_data  # Model data S3 location (Exact model artifact file)
            }
        ],
        "SupportedContentTypes": ["application/json"],
        "SupportedResponseMIMETypes": ["application/json"]
    },
)

# Print Model Package ARN
print(f"Model Package ARN: {create_model_package_response['ModelPackageArn']}")


Model Package ARN: arn:aws:sagemaker:us-east-1:284926681852:model-package/xgb-breastcancer-detection/1


In [27]:
# Describe the model package group
describe_model_package_group_response = sagemaker.describe_model_package_group(
    ModelPackageGroupName=model_package_group_name
)

print(describe_model_package_group_response)


{'ModelPackageGroupName': 'xgb-breastcancer-detection', 'ModelPackageGroupArn': 'arn:aws:sagemaker:us-east-1:284926681852:model-package-group/xgb-breastcancer-detection', 'ModelPackageGroupDescription': 'Model group for XGBoost models for breast cancer detection.', 'CreationTime': datetime.datetime(2024, 10, 1, 3, 0, 9, 823000, tzinfo=tzlocal()), 'CreatedBy': {'UserProfileArn': 'arn:aws:sagemaker:us-east-1:284926681852:user-profile/d-slh2nzpwelhn/dshalaby', 'UserProfileName': 'dshalaby', 'DomainId': 'd-slh2nzpwelhn', 'IamIdentity': {'Arn': 'arn:aws:sts::284926681852:assumed-role/LabRole/SageMaker', 'PrincipalId': 'AROAUEVXUEL6AW7HUZR4B:SageMaker'}}, 'ModelPackageGroupStatus': 'Completed', 'ResponseMetadata': {'RequestId': '4cf7ea5a-9cc4-44a9-8c3e-d634c893c7cb', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '4cf7ea5a-9cc4-44a9-8c3e-d634c893c7cb', 'content-type': 'application/x-amz-json-1.1', 'content-length': '627', 'date': 'Tue, 01 Oct 2024 03:18:51 GMT'}, 'RetryAttempts':

# Model Package

In [32]:
# Define model package group name and model data location
model_data = 's3://sagemaker-us-east-1-284926681852/DEMO-breast-cancer-prediction-xgboost-highlevel/output/xgb-2024-10-01-02-34-42/xgb-2024-10-01-02-34-42/output/model.tar.gz'
model_package_group_name = "xgb-breastcancer-detection"

# Define the inference image (Docker container for inference)
inference_image = image

# Create the model package
create_model_package_response = sagemaker.create_model_package(
    ModelPackageGroupName=model_package_group_name,
    ModelPackageDescription="Versioned XGBoost model for breast cancer detection.",
    ModelApprovalStatus="Approved",  # Or "PendingManualApproval" for manual review
    InferenceSpecification={
        "Containers": [
            {
                "Image": inference_image,  # Replace with your image
                "ModelDataUrl": model_data  # Model data S3 location
            }
        ],
        "SupportedContentTypes": ["application/json"],  # Input data type
        "SupportedResponseMIMETypes": ["application/json"]  # Output data type
    },
    
)

# Print the created model package ARN
model_package_arn = create_model_package_response['ModelPackageArn']
print(f"Model Package ARN: {model_package_arn}")


Model Package ARN: arn:aws:sagemaker:us-east-1:284926681852:model-package/xgb-breastcancer-detection/2


In [33]:
# Describe the model package to review the details
describe_model_package_response = sagemaker.describe_model_package(
    ModelPackageName=model_package_arn
)

# Print the description of the model package
print("Model Package Details:")
print(describe_model_package_response)


Model Package Details:
{'ModelPackageGroupName': 'xgb-breastcancer-detection', 'ModelPackageVersion': 2, 'ModelPackageArn': 'arn:aws:sagemaker:us-east-1:284926681852:model-package/xgb-breastcancer-detection/2', 'ModelPackageDescription': 'Versioned XGBoost model for breast cancer detection.', 'CreationTime': datetime.datetime(2024, 10, 1, 3, 28, 17, 747000, tzinfo=tzlocal()), 'InferenceSpecification': {'Containers': [{'Image': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.7-1', 'ImageDigest': 'sha256:f037aa7389a000dc611e723fb227d21a07cf495fd5e7bd8292a260ae101b5546', 'ModelDataUrl': 's3://sagemaker-us-east-1-284926681852/DEMO-breast-cancer-prediction-xgboost-highlevel/output/xgb-2024-10-01-02-34-42/xgb-2024-10-01-02-34-42/output/model.tar.gz'}], 'SupportedContentTypes': ['application/json'], 'SupportedResponseMIMETypes': ['application/json']}, 'ModelPackageStatus': 'Completed', 'ModelPackageStatusDetails': {'ValidationStatuses': [], 'ImageScanStatuses': []}, 'Certify

# Model Card

In [67]:
model_card_content = {
    "model_overview": {
        "model_description": "An XGBoost model trained to predict breast cancer based on diagnostic data.",
        "model_creator": "Dina",
        "model_owner": "Dina",
        "algorithm_type": "XGBoost",
        "problem_type": "Binary Classification",
        "model_artifact": [sm_estimator.model_data]
    },
    "training_details": {
        "training_observations": "The model was trained using SageMaker's built-in XGBoost algorithm.",
        "training_job_details": {
            "training_arn": sm_estimator.latest_training_job.describe()["TrainingJobArn"],
            "training_datasets": [train_data.config["DataSource"]["S3DataSource"]["S3Uri"]],
            "training_environment": {
                "container_image": [sm_estimator.training_image_uri()]
            },
            "hyper_parameters": [
                {"name": k, "value": str(v)} for k, v in sm_estimator.hyperparameters().items()
            ]
        }
    },
    "intended_uses": {
        "purpose_of_model": "Predict breastcancer",
        "intended_uses": "Used by doctors to aid in early cancer detection",
        "factors_affecting_model_efficiency": "Amount of data, quality of data",
        "risk_rating": "High",
        "explanations_for_risk_rating": "AI models are risky to use in clinical diagnostics",
    },
    "additional_information": {
        "caveats_and_recommendations": "Always consider human in the loop for cancer detection results",
        "ethical_considerations": "Ensure patient consent and annonmyzation",
        "custom_details": {
            "License": "NA"
        }
    }
}

model_card_content_str = json.dumps(model_card_content)

response = sagemaker_client.create_model_card(
    ModelCardName=model_card_name,
    Content=model_card_content_str,
    ModelCardStatus='Draft'
)

In [68]:
print(response)

{'ModelCardArn': 'arn:aws:sagemaker:us-east-1:284926681852:model-card/xgboost-breast-cancer-model', 'ResponseMetadata': {'RequestId': 'd1087a67-1aaf-414a-acae-edda3d7edb3f', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'd1087a67-1aaf-414a-acae-edda3d7edb3f', 'content-type': 'application/x-amz-json-1.1', 'content-length': '98', 'date': 'Tue, 01 Oct 2024 04:48:49 GMT'}, 'RetryAttempts': 0}}


In [71]:
# Provide your model card name
modelcardname = model_card_name 
sagemaker_client = boto3.client('sagemaker')
# Describe the model card
response = sagemaker_client.describe_model_card(
    ModelCardName=modelcardname
)

# Print the model card details
print("Model Card Details:")
print("===================")
print(f"Model Card Name: {response['ModelCardName']}")
print(f"Model Card Status: {response['ModelCardStatus']}")
print(f"Creation Time: {response['CreationTime']}")
print(f"Last Modified Time: {response['LastModifiedTime']}")
print(f"Model Card Content: {response['Content']}")


Model Card Details:
Model Card Name: xgboost-breast-cancer-model
Model Card Status: Draft
Creation Time: 2024-10-01 04:48:49.996000+00:00
Last Modified Time: 2024-10-01 04:48:49.996000+00:00
Model Card Content: {"model_overview": {"model_description": "An XGBoost model trained to predict breast cancer based on diagnostic data.", "model_creator": "Dina", "model_owner": "Dina", "algorithm_type": "XGBoost", "problem_type": "Binary Classification", "model_artifact": ["s3://sagemaker-us-east-1-284926681852/DEMO-breast-cancer-prediction-xgboost-highlevel/output/xgb-2024-10-01-02-34-42/xgb-2024-10-01-02-34-42/output/model.tar.gz"]}, "training_details": {"training_observations": "The model was trained using SageMaker's built-in XGBoost algorithm.", "training_job_details": {"training_arn": "arn:aws:sagemaker:us-east-1:284926681852:training-job/xgb-2024-10-01-02-34-42", "training_datasets": ["s3://sagemaker-us-east-1-284926681852/DEMO-breast-cancer-prediction-xgboost-highlevel/train"], "training

# Release resources

In [72]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

## Notebook CI Test Results

This notebook was tested in multiple regions. The test results are as follows, except for us-west-2 which is shown at the top of the notebook.

![This us-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/us-east-1/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with high level SDK.ipynb)

![This us-east-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/us-east-2/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with high level SDK.ipynb)

![This us-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/us-west-1/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with high level SDK.ipynb)

![This ca-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ca-central-1/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with high level SDK.ipynb)

![This sa-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/sa-east-1/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with high level SDK.ipynb)

![This eu-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-west-1/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with high level SDK.ipynb)

![This eu-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-west-2/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with high level SDK.ipynb)

![This eu-west-3 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-west-3/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with high level SDK.ipynb)

![This eu-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-central-1/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with high level SDK.ipynb)

![This eu-north-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-north-1/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with high level SDK.ipynb)

![This ap-southeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-southeast-1/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with high level SDK.ipynb)

![This ap-southeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-southeast-2/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with high level SDK.ipynb)

![This ap-northeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-northeast-1/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with high level SDK.ipynb)

![This ap-northeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-northeast-2/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with high level SDK.ipynb)

![This ap-south-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-south-1/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with high level SDK.ipynb)
